### 문제 2-1

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [4]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [7]:
output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template= 
    """
    사용자가 관심있는 분야는 {field}입니다.
    이 분야와 관련된 한국의 유명한 장소나 활동 5가지를 콤마로 구분된 리스트로 출력해주세요.
    {format_instructions}
    """,
    input_variables=["field"],
    partial_variables={"format_instructions": format_instructions},

)

model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

chain = prompt | model | output_parser

result = chain.invoke({"field":"음식"})

print(result)

['서울의 한옥마을', '전주 음식촌', '부산 광장시장', '제주 흑돼지 거리', '남도 음식거리']


### 문제 2-2

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain.schema import OutputParserException

from enum import Enum

In [19]:
class Sentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "중립"

enumParser = EnumOutputParser(enum = Sentiment)
format_instructions = enumParser.get_format_instructions()

template = """
당신은 영화 리뷰 감정 분석 전문가 입니다. 
다음 영화의 리뷰를 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.

영화 리뷰: "{review}"

{format_instructions}

"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions = format_instructions)

model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0  # 일관성을 위해 0으로 설정
)

fixing_parser = OutputFixingParser.from_llm(parser=enumParser, llm=model)

def run_sentiment_analysis(review_text):
    try:
        chain = prompt | model | fixing_parser
        result = chain.invoke({"review": review_text})
        return result, None
    except Exception as e:
        return None, f"오류: {str(e)}"

reviews = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."
]

for i, review in enumerate(reviews, 1):
    print(f"{i}. 리뷰: {review}")
    result, error = run_sentiment_analysis(review)
    print(result.value)


1. 리뷰: 이 영화 정말 재미없어요. 시간 낭비였습니다.
부정
2. 리뷰: 배우들의 연기가 훌륭하고 스토리도 감동적이었어요!
긍정
3. 리뷰: 그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.
중립


### 문제 2-3

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List

class Student(BaseModel):
    name: str = Field(description="이름")
    age: int = Field(description="나이")
    major: str = Field(description="전공")
    hobbies: List[str] = Field(description="취미")
    goal: str = Field(description="목표")

parser = PydanticOutputParser(pydantic_object=Student)

template = """
학생의 자기소개를 듣고 학생의 정보를  정리해주세요

자기소개: {introduction}

{format_instructions}

"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(
    format_instructions=parser.get_format_instructions()
)

model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

chain = prompt | model | parser

def 